In [1]:
import matplotlib.pyplot as plt
import numpy as np
import IPython
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import tensorflow_io as tfio
import functools
from pedalboard import load_plugin
from ReverberatorEstimator import layers, loss
import warnings
warnings.filterwarnings('ignore')
import time
import os
import librosa.display

2021-12-01 14:04:51.229535: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-01 14:04:51.229577: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
sample_rate = 48000
num_params = 28

In [3]:
target_audio = tfio.audio.AudioIOTensor("snap_wet.wav")
target_audio = target_audio.to_tensor()
target_audio = tf.slice(target_audio, begin=[0,0], size=[-1,1])
target_audio = tf.cast(target_audio, tf.float32) / 32768.0
target_audio = tf.squeeze(target_audio)
target_audio = tf.reshape(target_audio,(1, 96000))
input_audio = tfio.audio.AudioIOTensor("snap_dry.wav")
input_audio = input_audio.to_tensor()
input_audio = tf.slice(input_audio, begin=[0,0], size=[-1,1])
input_audio = tf.cast(input_audio, tf.float32) / 32768.0
input_audio = tf.squeeze(input_audio)
input_audio = tf.reshape(input_audio,(1, 96000))

2021-12-01 14:04:53.075528: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2021-12-01 14:04:53.312734: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-01 14:04:53.312862: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-01 14:04:53.312956: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-01 14:04:53.313045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dl

In [4]:
logmelgram = layers.LogMelgramLayer(1024, 256, 128, sample_rate, 0.0, sample_rate//2, 1e-6)
audio_time = tfkl.Input(shape=(96000,), name="audio_time")

x = logmelgram(audio_time)
x = tfkl.BatchNormalization(name="input_norm")(x)
encoder_model = tfk.applications.MobileNetV2(input_shape=(x.shape[1], x.shape[2], x.shape[3]), alpha=1.0,
                                            include_top=True, weights=None, input_tensor=None, pooling=None,
                                            classes=np.sum(num_params).item(), classifier_activation="sigmoid")

hidden = encoder_model(x)

parameter_model = tfk.models.Model(audio_time, hidden, name="parameter_model")

parameters = parameter_model(audio_time)

vstlayer = layers.VSTProcessor("../Reverberator.vst3", sample_rate)
output = vstlayer([audio_time, parameters])

model = tfk.models.Model(audio_time, output, name="full_model")

spectral_loss = loss.multiScaleSpectralLoss(sr=sample_rate)

optimizer = tfk.optimizers.Adam(learning_rate=0.01)

checkpoint_dir = './training_checkpoints'

model.compile(optimizer=optimizer, loss=spectral_loss, metrics=['mae'], run_eagerly=True)

No protocol specified
No protocol specified
No protocol specified
No protocol specified


In [5]:
parameter_model.summary()
model.summary()

Model: "parameter_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
audio_time (InputLayer)      [(None, 96000)]           0         
_________________________________________________________________
log_melgram_layer (LogMelgra (None, 372, 128, 1)       0         
_________________________________________________________________
input_norm (BatchNormalizati (None, 372, 128, 1)       4         
_________________________________________________________________
mobilenetv2_1.00_372 (Functi (None, 28)                2293276   
Total params: 2,293,280
Trainable params: 2,259,166
Non-trainable params: 34,114
_________________________________________________________________
Model: "full_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
audio_time (Inpu

In [6]:
# Restore from latest checkpoint
model.load_weights(checkpoint_dir)

2021-12-01 14:04:55.227282: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ./training_checkpoints: Failed precondition: training_checkpoints; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [7]:
audio_pre = (model.call(input_audio)).numpy()[0]
old_params = parameter_model(input_audio).numpy()[0]
print(old_params)

[0.5000084  0.4999778  0.49988297 0.5000118  0.49991092 0.5001264
 0.49997008 0.49988535 0.5000775  0.49988642 0.49998417 0.5000142
 0.49986145 0.50010145 0.49993053 0.50002843 0.49997926 0.50013983
 0.50003535 0.49998054 0.49981967 0.49988514 0.5001355  0.4999809
 0.4999524  0.49993125 0.50000525 0.4999644 ]


No protocol specified
No protocol specified


In [11]:
model_cp = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                  factor=0.2,
                                  patience=10,
                                  cooldown=0,
                                  verbose=1,
                                  mode='min',
                                  min_lr=0.0000016)

In [ ]:
start_time = time.time()
history = model.fit(input_audio, target_audio, verbose=1, epochs=100,
         callbacks=[model_cp,reduce_lr])
print("Training took %d seconds" % (time.time() - start_time))

Epoch 1/100


No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified


1/1 [==============================] - 2s 2s/step - loss: 0.2002 - mae: 0.0070

Epoch 00001: loss did not improve from 0.19949
Epoch 2/100


No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified


1/1 [==============================] - 2s 2s/step - loss: 0.1984 - mae: 0.0069

Epoch 00002: loss improved from 0.19949 to 0.19841, saving model to ./training_checkpoints
INFO:tensorflow:Assets written to: ./training_checkpoints/assets
Epoch 3/100


No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified


1/1 [==============================] - 2s 2s/step - loss: 0.1984 - mae: 0.0069

Epoch 00003: loss did not improve from 0.19841
Epoch 4/100


No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified
No protocol specified


1/1 [==============================] - 2s 2s/step - loss: 0.1974 - mae: 0.0069

Epoch 00004: loss improved from 0.19841 to 0.19744, saving model to ./training_checkpoints


In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
output_audio = model(input_audio)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(7,10))
ax[0].plot(audio_pre)
D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_pre)), ref=np.max)
img = librosa.display.specshow(D, y_axis='linear', x_axis='time',
                               sr=sample_rate, ax=ax[1])

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))
ax[0,0].plot(output_audio.numpy()[0])
D = librosa.amplitude_to_db(np.abs(librosa.stft(output_audio.numpy()[0])), ref=np.max)
img = librosa.display.specshow(D, y_axis='linear', x_axis='time',
                               sr=sample_rate, ax=ax[1,0])
ax[0,1].plot(target_audio.numpy()[0])
D = librosa.amplitude_to_db(np.abs(librosa.stft(target_audio.numpy()[0])), ref=np.max)
img = librosa.display.specshow(D, y_axis='linear', x_axis='time',
                               sr=sample_rate, ax=ax[1,1])

In [ ]:
IPython.display.Audio(output_audio, rate=sample_rate, autoplay=True)

In [ ]:
IPython.display.Audio(audio_pre, rate=sample_rate)

In [ ]:
IPython.display.Audio(target_audio, rate=sample_rate)

In [ ]:
# Dump values to .csv files
np.savetxt("output_audio.csv", output_audio.numpy()[0], delimiter=",")
np.savetxt("target_audio.csv", target_audio.numpy()[0], delimiter=",")

In [ ]:
parameters = parameter_model(input_audio)

In [ ]:
params = parameters.numpy()[0]
filter_c = 1
for i in range(num_params):
    if i < 4:
        print("b_%i = %f" % (i, params[i]))
    elif i < 8:
        print("c_%i = %f" % (i-4, params[i]))
    else:
        j = (i-8) % 5
        
        if j is 0:
            print("\nFilter %i:" % filter_c)
            filter_c = filter_c + 1
            print("c_hp = %f" % params[i])
        elif j is 1:
            print("c_bp = %f" % params[i])
        elif j is 2:
            print("c_lp = %f" % params[i])
        elif j is 3:
            print("g = %f" % params[i])
        elif j is 4:
            print("R = %f" % params[i])

In [ ]:
param_diff = old_params - params
print(param_diff)
plt.stem(param_diff)
plt.ylim([-1, 1])